In [7]:
!pip install tensorflow_hub

In [8]:
!pip install tensorflow_text

In [3]:
import numpy as np
import pandas as pd
import sklearn.preprocessing 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import string
import nltk
from nltk.corpus import stopwords
from textblob import Word
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import wordcloud
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split 
from keras.utils import np_utils
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
sns.set()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

In [6]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TUKL summer internship/week 2/Supreme Court Data 5.csv')
df.head()

,Case Files,Label,Category
0,2017 P T D 1481\n[Supreme Court of Pakistan]\n...,Non-Bailable,Against Federation
1,2015 S C M R 365\n[Supreme Court of Pakistan]\...,Bailable,illegal appointments
2,2016 S C M R 1420\n[Supreme Court of Pakistan]...,Bailable,Theft/Fraud
3,2016 S C M R 662\n[Supreme Court of Pakistan]\...,Bailable,Theft/Fraud
4,P L D 2012 Supreme Court 649\nPresent: Mian Sh...,Bailable,illegal appointments


In [9]:
df1 = df[['Case Files','Label']]
df1.isnull().sum()

Case Files    0
Label         0
dtype: int64

In [10]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [11]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

In [12]:
def cleaning(df, stop_words):

    df['Case Files'] = df['Case Files'].apply(lambda x: ' '.join(x.lower() for x in x.split())) 

                               

    # Replacing the digits/numbers

    df['Case Files'] = df['Case Files'].str.replace('d', '')

            

    # Removing stop words

    df['Case Files'] = df['Case Files'].apply(lambda x: ' '.join(x for x in x.split() if x not in stop_words))

    #lower casing
    df['Case Files'] = df['Case Files'].str.lower()

    #punctuation removal         
    df["Case Files"] = df["Case Files"].apply(lambda text: remove_punctuation(text))
    # Lemmatization

    df['Case Files'] = df['Case Files'].apply(lambda x: ' '.join([Word(x).lemmatize() for x in x.split()]))

            

    return df

stop_words = stopwords.words('english')

df1 = cleaning(df1, stop_words)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [13]:
df2 = df1.rename(columns={'Case Files': 'Case_Files'})
df2.head()

,Case_Files,Label
0,2017 p 1481 supreme court pakistan present mia...,Non-Bailable
1,2015 c r 365 supreme court pakistan present na...,Bailable
2,2016 c r 1420 supreme court pakistan present a...,Bailable
3,2016 c r 662 supreme court pakistan present an...,Bailable
4,p l 2012 supreme court 649 present mian shakir...,Bailable


In [14]:
#tokenizer = Tokenizer(num_words=500, split=' ') 
#tokenizer.fit_on_texts(df2['Case_Files'].values)
#X = tokenizer.texts_to_sequences(df2['Case_Files'].values)
#X = pad_sequences(X)

In [15]:
df2['encoded']=df2['Label'].apply(lambda x: 1 if x=='Bailable' else 0)
df2.head()

,Case_Files,Label,encoded
0,2017 p 1481 supreme court pakistan present mia...,Non-Bailable,0
1,2015 c r 365 supreme court pakistan present na...,Bailable,1
2,2016 c r 1420 supreme court pakistan present a...,Bailable,1
3,2016 c r 662 supreme court pakistan present an...,Bailable,1
4,p l 2012 supreme court 649 present mian shakir...,Bailable,1


In [16]:
df2['encoded']

0     0
1     1
2     1
3     1
4     1
     ..
61    1
62    0
63    0
64    1
65    1
Name: encoded, Length: 66, dtype: int64

In [17]:
df3=df2.copy()
df4=df2.copy()
df5=df2.copy()
df6=df2.copy()
df7=pd.concat([df2,df3,df4,df5,df6])
df8=df7.copy()
df9=df7.copy()
df10=df7.copy()
df11=df7.copy()
df12=pd.concat([df8,df9,df10,df11])
df2=df12
df2.shape

(1320, 3)

In [18]:
from sklearn.model_selection import train_test_split#stratify ensures balance between spam and no spam in test and train

X_train, X_test, y_train, y_test = train_test_split(df2['Case_Files'],df2['encoded'],test_size=0.2, stratify=df2['encoded'])

In [19]:
y_test.value_counts()

1    172
0     92
Name: encoded, dtype: int64

In [20]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [21]:
def get_sentence_embeding(sentences):#generates embedding vector for sentence
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [22]:
preprocessor = hub.load(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

In [23]:
#so far we have used sequential models but now we are using BERT which is a functional model.
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')#name can be anything #input shape variable #data type string
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [24]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           text[0][0]                       
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'default': (None, 7 109482241   keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [25]:
len(X_train)

1056

In [26]:
X_train.dtype

dtype('O')

In [27]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [28]:
model.fit(X_train, y_train,batch_size=128, epochs=5,shuffle=True)

Epoch 1/5
9/9 [==============================] - 473s 51s/step - loss: 0.6613 - accuracy: 0.6184
Epoch 2/5
9/9 [==============================] - 462s 51s/step - loss: 0.6363 - accuracy: 0.6572
Epoch 3/5
9/9 [==============================] - 462s 51s/step - loss: 0.6335 - accuracy: 0.6847
Epoch 4/5
9/9 [==============================] - 461s 51s/step - loss: 0.6329 - accuracy: 0.7121
Epoch 5/5
9/9 [==============================] - 462s 51s/step - loss: 0.6237 - accuracy: 0.7008


In [29]:
model.evaluate(X_test, y_test)

9/9 [==============================] - 120s 13s/step - loss: 0.6239 - accuracy: 0.7197


[0.6239221096038818, 0.7196969985961914]